In [1]:
pip install cryptography pynacl

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   -------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\dubey\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import hashlib
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import (
    Encoding, PublicFormat, PrivateFormat, NoEncryption, load_pem_private_key, load_pem_public_key
)
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from nacl.signing import SigningKey, VerifyKey
from nacl.exceptions import BadSignatureError
from base64 import urlsafe_b64encode, urlsafe_b64decode
from typing import Dict, Any, List


In [3]:
def generate_voter_keys():
    # Ed25519 keys for signature
    signing_key = SigningKey.generate()
    verify_key = signing_key.verify_key
    # Save as hex or bytes
    return signing_key, verify_key

def generate_server_ec_keypair():
    # ECC for key exchange (P-256 curve)
    private_key = ec.generate_private_key(ec.SECP256R1())  # Use SECP256K1 for Bitcoin-like, R1 for general
    public_key = private_key.public_key()
    return private_key, public_key

def serialize_ec_public_key(pubkey):
    return pubkey.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)

def deserialize_ec_public_key(bytes_data):
    return load_pem_public_key(bytes_data)


In [4]:
def ecies_encrypt(aes_key: bytes, recipient_pubkey):
    # ECDH: ephemeral keys for forward secrecy
    ephemeral_key = ec.generate_private_key(ec.SECP256R1())
    shared_secret = ephemeral_key.exchange(ec.ECDH(), recipient_pubkey)
    # Derive symmetric key (use HKDF for robust KDF)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"ecies-exchange"
    ).derive(shared_secret)
    # Use derived key to encrypt aes_key for transmission
    f = Fernet(urlsafe_b64encode(derived_key))
    encrypted_key = f.encrypt(aes_key)
    # Send both encrypted_key and ephemeral public key!
    ephemeral_pub = ephemeral_key.public_key()
    return encrypted_key, serialize_ec_public_key(ephemeral_pub)

def ecies_decrypt(encrypted_aes_key: bytes, ephemeral_pub_bytes: bytes, server_private_key):
    ephemeral_pub = deserialize_ec_public_key(ephemeral_pub_bytes)
    shared_secret = server_private_key.exchange(ec.ECDH(), ephemeral_pub)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"ecies-exchange"
    ).derive(shared_secret)
    f = Fernet(urlsafe_b64encode(derived_key))
    aes_key = f.decrypt(encrypted_aes_key)
    return aes_key


In [5]:
def create_vote(vote_text: str, server_pubkey, voter_signing_key: SigningKey):
    # Symmetric key and vote encryption
    sym_key = Fernet.generate_key()  # 32 bytes
    f = Fernet(sym_key)
    vote_enc = f.encrypt(vote_text.encode())
    # ECC encrypt symmetric key (ECIES)
    encrypted_key, ephemeral_pub_bytes = ecies_encrypt(sym_key, server_pubkey)
    # Sign encrypted vote (Ed25519)
    signature = voter_signing_key.sign(vote_enc).signature
    return {
        "vote_encrypted": urlsafe_b64encode(vote_enc).decode(),
        "key_encrypted": urlsafe_b64encode(encrypted_key).decode(),
        "ephemeral_pub": urlsafe_b64encode(ephemeral_pub_bytes).decode(),
        "signature": urlsafe_b64encode(signature).decode()
    }


In [6]:
def verify_and_store_vote(
    vote_packet: Dict[str, Any],
    voter_verify_key: VerifyKey,
    server_ec_private_key,
    prev_hash: str
):
    try:
        # Decode fields
        vote_enc = urlsafe_b64decode(vote_packet["vote_encrypted"])
        key_enc = urlsafe_b64decode(vote_packet["key_encrypted"])
        ephemeral_pub = urlsafe_b64decode(vote_packet["ephemeral_pub"])
        signature = urlsafe_b64decode(vote_packet["signature"])
        # Signature verification
        voter_verify_key.verify(vote_enc, signature)
        # Decrypt symmetric key with ECC
        aes_key = ecies_decrypt(key_enc, ephemeral_pub, server_ec_private_key)
        f = Fernet(aes_key)
        vote_plain = f.decrypt(vote_enc).decode()
        # Hash chain
        prev_hash_bytes = prev_hash.encode()
        vote_hash = hashlib.blake2b(vote_enc + prev_hash_bytes, digest_size=32).hexdigest()
        return {
            "vote_plain": vote_plain,
            "vote_hash": vote_hash
        }
    except BadSignatureError:
        print("Signature verification failed!")
        return None
    except Exception as e:
        print(f"Vote processing failed: {e}")
        return None


In [7]:
def main():
    print("=== Secure Voting System CLI Example ===")
    # Server EC key
    server_priv, server_pub = generate_server_ec_keypair()

    NUM_VOTERS = 3
    voters = []
    votes = []
    voter_registry = {}

    # Setup voters, key registry
    for i in range(NUM_VOTERS):
        sk, vk = generate_voter_keys()
        voters.append({"sign": sk, "verify": vk})
        voter_registry[f"voter{i+1}"] = vk

    # Voting
    for idx, voter in enumerate(voters):
        vote_text = input(f"Voter {idx+1}, enter your vote (e.g., 'Alice', 'Bob'): ")
        vote_packet = create_vote(
            vote_text,
            server_pub,
            voter["sign"]
        )
        votes.append((f"voter{idx+1}", vote_packet))

    # Tally votes with hash chain
    print("\nVote Tally Begin (with hash chain)")
    results = {}
    prev_hash = "0" * 64
    chain: List[Dict[str, Any]] = []
    for voter_id, vote_packet in votes:
        out = verify_and_store_vote(
            vote_packet, voter_registry[voter_id], server_priv, prev_hash
        )
        if out:
            print(f"Vote accepted: {out['vote_plain']}")
            results[out['vote_plain']] = results.get(out['vote_plain'], 0) + 1
            prev_hash = out['vote_hash']
            chain.append({"vote": out['vote_plain'], "hash": out['vote_hash']})
        else:
            print(f"Vote from {voter_id} discarded!")
    print("\nResults Tally:")
    for k, v in results.items():
        print(f"{k}: {v}")
    print("\nHash Chain Log:")
    for ch in chain:
        print(f"{ch['vote']} -> {ch['hash']}")

if __name__ == "__main__":
    main()


=== Secure Voting System CLI Example ===

Vote Tally Begin (with hash chain)
Vote accepted: Alice
Vote accepted: Bob
Vote accepted: Alice

Results Tally:
Alice: 2
Bob: 1

Hash Chain Log:
Alice -> 954b8381171fe9480301582c6abfbc03c0c8eb095795eabd9c78f49451e0beb2
Bob -> 011a2fa6adbf56d535e1c36c3440cfec0f84a7579064d42d7e6fb7eadd665974
Alice -> 0d765245da26e86d581729ce01ec223e61260fbc8dea6011e7c7c25a6c24739e
